# Live Sandbox

In [ ]:
import pandas as pd
import math
from scipy import interpolate

In [ ]:
# Definition of constants and variables

# Some common physical constants, mainly used to calculate `maxFlux`
## (the upper bound of energy flux received by a planet that allows it
## to be habitable according to Pierrehumbert (2015))

BIGG       = 6.67428e-11       # Gravitational constant
PI         = 3.1415926535
A          = 0.7344            # Pierrehumbert's Constant
SB         = 5.670373e-8       # Stefan-Boltzmann Constant
LH2O       = 2.425e6           # Latent Heat Capacity of Water
RGAS       = 461.5             # Universal Gas Constant
PLINE      = 1e4               
PREF       = 610.616           # Reference Pressure
TREF       = 273.13            # Reference Temperature
K0         = 0.055             # A constant in Runaway Greenhouse calculation


# Boundaries for albedo values:
ALBMINELSE = 0.05              # General lower bound
ALBMAXELSE = 0.8               # General upper bound
ALBMING    = 0.25              # Lower bound for planets orbiting G-type stars
ALBMAXM    = 0.35              # Upper bound for planets orbiting M-type stars


# The counterpart to maxFlux
## (unlike maxFlux, MINFLUX is a constant that does not depend on a planet's properties)
MINFLUX    = 67

# Definitions of units of measurement,
## mainly used to convert [exoplanet.org](exoplanet.org) data into SI units:
MEARTH     = 5.972186e24       # Earth mass in kilograms
REARTH     = 6378100           # Earth's radius in meters
S0         = 1362              # Solar constant in watts per square meter
MSUN       = 1.988416e30       # Solar mass in kilograms
RSUN       = 6.957e8           # Solar radius in meters
LSUN       = 3.828e26          # Solar luminosity in watts
RJUP       = 7.1492e7          # Jovian radius in meters
MJUP       = 1.8982e27         # Jovian mass in kilograms
AU         = 1.496e11          # The astronomical unit in meters

In [ ]:
# Definition of functions

# 1) A function to calculate the probability distribution of orbital eccentricity
def pofe(ecc,mu,sigma):
    return ((sigma*math.sqrt(2*math.pi))**(-1))*math.exp(-(((ecc-mu)**2)/(2*sigma**2)))/1000

# 2) A function to calculate the probability of a planet's terrestriality
def fp_ter(mPlanet,rPlanet,exoName):
    # Convert the unit to Earth's masses and radii
    mPlanet = mPlanet/MEARTH 
    rPlanet = rPlanet/REARTH
    
    # Calculate mu1
    ## Initialize mu1 value to 0
    mu1 = 0.0       
    
    # Initialize temporary variables to hold a mass/radius value
    ## from the (i-1)th row of the ZS table
    mZSimin1 = 0
    rZSimin1 = 0

    # This block iterates through the the 'M-Pure-MgSiO3' column
    #to find the bracket that contains mPlanet value
    for i in rowNum:
        # Initialize temporary variables to hold a mass/radius value
        #from the i-th row of the ZS table
        mZSi = zs.loc[i, "M-PureMgSiO3"]
        rZSi = zs.loc[i, "R-PureMgSiO3"]
        
        # Comparing mPlanet to the current value of mZSi
        if mPlanet == mZSi:
            mu1 = rZSi
            break                          
        elif mPlanet > mZSi:               
            mZSimin1 = mZSi                
            rZSimin1 = rZSi                  
        else: # if mPlanet < mZSi --> we have found the correct bracket
            f = interpolate.interp1d(zs.loc[(i-1):(i), "M-PureMgSiO3"], zs.loc[(i-1):(i), "R-PureMgSiO3"], kind='linear', assume_sorted=True)
            mu1 = f(mPlanet)
            break

    # Calculate mu2
    mu2 = 0.0
    mZSimin1 = 0
    rZSimin1 = 0
    for i in rowNum:
        mZSi = zs.loc[i, "M-MgSiO3-H2O-5050"]
        rZSi = zs.loc[i, "R-MgSiO3-H2O-5050"]
        if mPlanet == mZSi:
            mu2 = rZSi
            break
        elif mPlanet > mZSi:
            mZSimin1 = mZSi
            rZSimin1 = rZSi
        else: 
            f = interpolate.interp1d(zs.loc[(i-1):(i), "M-MgSiO3-H2O-5050"], zs.loc[(i-1):(i), "R-MgSiO3-H2O-5050"], kind='linear', assume_sorted=True)
            mu2 = f(mPlanet)
            break

    # Calculate sigma1
    sigma1 = (mu2-mu1)/3
    
    # Calculate the terrestrial probability
    p_ter = 0
    if rPlanet <= mu1:
        p_ter = 1
    elif rPlanet >= mu2:
        p_ter = 0
    else: # uses a pseudo-gaussian function
        p_ter = math.exp(-(0.5)*((rPlanet-mu1)/sigma1)**2)
    return p_ter

In [ ]:
# Data input
## Import exoplanet data from a CSV file into a pandas dataframe
exo = pd.read_csv (r'exoplanets.csv', low_memory=False)

# Set the column with the header NAME to be used as an index to identify row 
exo = exo.set_index("NAME", drop = False)

# Extract names of planets as a list (to be used as a calling list)
exoList = pd.DataFrame(exo, columns=['NAME'])
exoList = exoList['NAME'].values.tolist()

In [5]:
# Zeng-Sasselov boundaries input

## Import CSV of Zeng & Sasselov boundaries
zs = pd.read_csv (r'zsboundaries.csv')

## Set index using the RowNum column
zs = zs.set_index("RowNum", drop = False)

## Extract the column "RowNum" as a list (to be used as a calling list)
rowNum = pd.DataFrame(zs, columns=['RowNum'])
rowNum = rowNum['RowNum'].values.tolist()

In [13]:
# Main subroutine to determine the habitability index value

habIndex = []
habIndexWithName = []
habIndexNotZero = []
i = 1

for exoName in exoList:
    print(i, "-", exoName)
    i = i+1
    
    # Extract data of individual planets
    # HOST STAR PROPERTIES
    # Stellar radius (in solar radii)
    rStar = exo.loc[exoName, "RSTAR"]
    ## Convert to SI
    rStar = rStar*RSUN
    
    
    # Stellar temperature (in Kelvin)
    teffStar = exo.loc[exoName, "TEFF"]
    
    
    # Stellar luminosity
    luminosity = 4*math.pi*rStar*rStar*SB*teffStar**4
    
    # PLANET PROPERTIES   
    # Planetary radius (in Jovian radii)
    rPlanet = exo.loc[exoName, "R"]
    ## If R is not available, calculate it from  transit depth
    if math.isnan(rPlanet) == 1:
        depth = exo.loc[exoName, "DEPTH"]
        if math.isnan(depth) == 1:
            continue
        rPlanet = math.sqrt(depth)*rStar
    ## Convert to SI
    rPlanet = rPlanet*RJUP
    
    
    # Planetary mass (in Jovian masses)
    mPlanet = exo.loc[exoName, "MASS"] 
    ## If MASS is not available, calculate it from a common scaling law
    ### from the original HITE
    if math.isnan(mPlanet) == 1:
        if rPlanet/REARTH <= 1:
            mPlanet = ((rPlanet/REARTH)**3.268)*MEARTH
        elif rPlanet/REARTH > 1:
            mPlanet = ((rPlanet/REARTH)**3.65)*MEARTH
    ## Convert to SI
    mPlanet = mPlanet*MJUP
    
   
    
    
    # Surface planet gravity (in SI)
    surfGrav = BIGG*mPlanet/(rPlanet**2)   
    
  
    
    # ORBITAL PROPERTIES
    
    # Orbital eccentricity
    ecc = exo.loc[exoName, "ECC"]
    
    

    # Measurement uncertainty of orbital eccentricity    
    ## Upper bound (relative from E)
    eccUpRel = exo.loc[exoName, "ECCUPPER"]
    ### If measurement uncertainty is not available, assign it as 0.01
    if math.isnan(eccUpRel) == 1 or eccUpRel == 0:
        eccUpRel = 0.01
    ### Upper bound (absolute)
    eccUpper = ecc + eccUpRel
    
   
    ## Lower bound (relative from E)
    eccLowRel = exo.loc[exoName, "ECCLOWER"]
    ### If measurement uncertainty is not available, assign it as 0.01
    if math.isnan(eccLowRel) == 1:
        if ecc != 0:
            eccLowRel = 0.01
        else:
            eccLowRel = 0
    
    ###Lower bound (absolute)
    eccLower = ecc - eccLowRel
    

    
    # Orbital semi-major axis (in AU)
    semiAxis = exo.loc[exoName, "A"]      
    ## Convert to SI
    semiAxis = semiAxis*AU
    
    
    
    
    # Calculate the upper and lower bounds of F_OLR [...]
    ## that would allow for surface liquid water to exist
    pStar = PREF*math.exp(LH2O/(RGAS*TREF))
    # Upper bound: maximum F_OLR
    maxFlux = A*SB*(LH2O/(RGAS*math.log(pStar*math.sqrt(K0/(2*PLINE*surfGrav)))))**4
    # Lower bound: minimum F_OLR is the constant MINFLUX
    minFlux = MINFLUX
    
    # Probability of the planet being terrestrial
    p_ter = fp_ter(mPlanet,rPlanet,exoName)
        
    
    # Albedo (new)
    ## Boundaries
    albMin = ALBMINELSE
    albMax = ALBMAXELSE
    ## Special conditions
    ### For planets with M-type host star
    if teffStar >= 2300 and teffStar <=3800:
        albMax = ALBMAXM
    ### For planets with G-type host star
    elif teffStar >= 5370 and teffStar <=5980:
        albMin = ALBMING
        
        
    
    # Calculate F_OLR
    ## Albedo increments
    da = 0.01
    ## Eccentricity increments
    de = 0.01
    ## Sum of pofe (probability of eccentricity);
    ### (is used to normalize the index value, later)
    ### Initialized to 0
    pofeSum = 0
    ### Sum of how many instances of F_OLR meets the requirements for
    #### the planet to have surface liquid water. Each instances will then be
    #### multiplied by the probability of its eccentricity (pofe)
    ### Initialized to 0
    habFact = 0
    ### Incoming stellar radiation (instellation)
    flux0 = luminosity/(16*math.pi*semiAxis*semiAxis)

    # Calculate the habitability index
    ## Iterate through the albedo & eccentricity 2D matrix
    a = albMin
    while a < albMax:
        e = eccLower
        while e < eccUpper:
            flux = flux0*(1-a)/math.sqrt(1-e*e)
            pofeSum = pofeSum + pofe(e, ecc, eccUpRel)
            if flux < maxFlux and flux > MINFLUX:
                habFact = habFact + pofe(e, ecc, eccUpRel)
            e = e + de
        a = a + da   
    
    if ecc > 0.8:
        H = 0.0
    elif pofeSum != 0:
        H = (habFact/pofeSum)*p_ter
    else: # in the case of error; might be better to replace this with a throw exception statement
        H = 0.0
    
    habIndex.append(H)
    habIndexWithName.extend([exoName, ",", H])
    
    if H > 0:
        habIndexNotZero.extend([exoName+ ","+str(H)])
    print(eccLower)    
    print(H)

1 - Kepler-107 d
nan
0.0
2 - Kepler-1049 b
0.0
0.0
3 - Kepler-813 b
0.0
0.0
4 - Kepler-427 b
0.0
0.0
5 - Kepler-1056 b
0.0
0.0
6 - Kepler-1165 b
0.0
0.0
7 - Kepler-1104 b
0.0
0.0
8 - WASP-14 b
0.088
0.0
9 - Kepler-50 b
nan
0.0
10 - NN Ser d
11 - WASP-105 b
0.0
0.0
12 - GJ 625 b
13 - Kepler-1279 b
0.0
0.0
14 - Kepler-1599 b
0.0
0.011431450472113318
15 - Kepler-20 b
0.0
0.0
16 - HAT-P-27 b
0.0
0.0
17 - Kepler-181 b
nan
0.0
18 - HD 116029 b
19 - Kepler-207 b
nan
0.0
20 - Kepler-1156 b
0.0
0.0
21 - Kepler-1512 b
0.0
0.0
22 - Kepler-787 b
0.0
0.0
23 - Kepler-528 b
0.0
0.0
24 - HD 219828 b
25 - Kepler-480 b
0.0
0.0
26 - Kepler-1567 b
0.0
0.0
27 - HD 3167 b
0.0
nan
28 - Kepler-1390 b
0.0
0.0
29 - Kepler-1642 b
0.0
0.0
30 - Kepler-11 c
0.2364256999999999
0.0
31 - Kepler-871 b
0.0
0.0
32 - Kepler-1131 b
0.0
0.0
33 - Kepler-705 b
0.0
0.0
34 - HD 1237 b
35 - WASP-21 b
0.0
0.0
36 - Kepler-284 c
nan
0.0
37 - Kepler-186 c
0.03
0.0
38 - HD 4308 b
39 - Kepler-239 b
nan
0.0
40 - Kepler-651 b
0.0
0.0
41

C:\Users\Zulfa\AppData\Local\Temp\ipykernel_11352\3739224329.py:140: RuntimeWarning: divide by zero encountered in double_scalars
  flux0 = luminosity/(16*math.pi*semiAxis*semiAxis)


0.0
60 - Kepler-1646 b
0.0
0.0
61 - HATS-11 b
0.0
0.0
62 - HD 24040 b
63 - Kepler-359 c
nan
0.0
64 - Kepler-211 c
nan
0.0
65 - Kepler-967 b
0.0
0.0
66 - Kepler-1574 b
0.0
0.0
67 - Kepler-1157 b
0.0
0.0
68 - Kepler-549 b
0.0
0.0
69 - Kepler-1510 b
0.0
0.0
70 - HD 59686 A b
71 - Kepler-1002 b
0.0
0.0
72 - Kepler-294 c
nan
0.0
73 - Kepler-263 c
nan
0.0
74 - Kepler-349 b
nan
0.0
75 - Kepler-1563 b
0.0
0.0
76 - Kepler-1611 b
0.0
0.0
77 - Kepler-1284 b
0.0
0.0
78 - HD 200964 b
79 - Kepler-706 b
0.0
0.0
80 - HD 147018 b
81 - HD 218566 b
82 - HD 45364 b
83 - Kepler-1177 b
0.0
0.0
84 - Kepler-1364 b
0.0
0.0
85 - Kepler-653 b
0.0
0.0
86 - Kepler-553 b
0.0
0.0
87 - Kepler-361 b
nan
0.0
88 - Kepler-318 c
nan
0.0
89 - WASP-140 b
0.0435
0.0
90 - Kepler-372 b
nan
0.0
91 - Kepler-197 e
nan
0.0
92 - HD 141937 b
93 - K2-11 b
0.0
0.0
94 - Kepler-405 c
nan
0.0
95 - Kepler-104 d
nan
0.0
96 - Kepler-702 b
0.0
0.0
97 - Kepler-26 d
nan
0.0
98 - Kepler-159 b
nan
0.0
99 - HD 156668 b
100 - MOA-2009-BLG-387L b
1

C:\Users\Zulfa\AppData\Local\Temp\ipykernel_11352\3739224329.py:103: RuntimeWarning: divide by zero encountered in double_scalars
  maxFlux = A*SB*(LH2O/(RGAS*math.log(pStar*math.sqrt(K0/(2*PLINE*surfGrav)))))**4


0.0
0.0
150 - Kepler-295 d
nan
0.0
151 - Kepler-337 b
nan
0.0
152 - HIP 67537 b
153 - HD 177830 b
154 - Kepler-516 b
0.0
0.0
155 - Kepler-1562 b
0.0
0.0
156 - Kepler-226 d
nan
0.0
157 - K2-99 b
0.15
0.0
158 - HD 42012 b
159 - HD 38283 b
160 - Kepler-386 b
nan
0.0
161 - Kepler-922 b
0.0
0.0
162 - HD 217786 b
163 - Kepler-363 c
nan
0.0
164 - Kepler-687 b
0.0
0.0
165 - Kepler-1457 b
0.0
0.0
166 - CoRoT-24 c
0.0
0.0
167 - Kepler-567 b
0.0
0.0
168 - Kepler-1453 b
0.0
0.0
169 - Kepler-24 d
nan
0.0
170 - HD 32518 b
171 - Kepler-980 b
0.0
0.0
172 - Kepler-472 b
0.0
0.0
173 - Kepler-402 c
nan
0.0
174 - Kepler-435 b
0.037000000000000005
0.0
175 - WASP-18 b
0.007529999999999999
0.0
176 - HD 145934 b
177 - CoRoT-6 b
0.0
0.0
178 - Kepler-127 c
nan
0.0
179 - Kepler-839 b
0.0
0.0
180 - Kepler-203 c
nan
0.0
181 - Kepler-1234 b
0.0
0.0
182 - Kepler-1127 b
0.0
0.0
183 - Kepler-732 c
0.0
0.0
184 - HD 73267 b
185 - HD 147513 b
186 - K2-28 b
0.0
0.0
187 - Kepler-1528 b
0.0
0.0
188 - Kepler-893 b
0.0
0.0
18

In [14]:
for i in habIndexNotZero:
    print(i)

Kepler-1599 b,0.011431450472113318
Kepler-440 b,0.019990687758356843
Kepler-1638 b,0.013956575564003683
Kepler-1605 b,0.05333333333333339
TRAPPIST-1 e,1.0
Kepler-1450 b,0.040627126793318404
TRAPPIST-1 f,0.7668910798209205
Kepler-1410 b,0.05782394210427924
Kepler-186 f,0.7810451825750875
TRAPPIST-1 g,0.21768360577007176
Kepler-1389 b,0.03165713362731174
Kepler-1229 b,1.0
Kepler-1459 b,0.09333333333333345
KOI-4427 b,0.007618587540148847
Kepler-1544 b,0.09302112425470994
TRAPPIST-1 d,0.0860513734597732
Kepler-69 c,0.18015940504014624
Kepler-1185 b,0.07272727272727271
Kepler-442 b,0.9466666666666669
LHS 1140 b,1.0
Kepler-441 b,0.22378053033592013
KOI 2124.01,0.28000000000000036
KOI 2290.01,0.016650533139208633
KOI 2401.01,0.04000000000000004
KOI 314.02,0.10666666666666678
KOI 7621.01,0.013262059008795237
KOI 4242.01,0.012487899854406454
KOI 4550.01,0.028696639197440525
KOI 4555.01,0.6181818181818183
KOI 7706.01,0.32000000000000045
KOI 7716.01,0.45454545454545464
KOI 4878.01,0.8400000000000

In [8]:
# Append the result (planet's name and index value) to a .txt file in the same folder 
#with open('out.txt','w') as f:
#    i = 1
#    for a in habIndexWithName:
#        if i == 3:
#            print(a, file=f)
#            i = 1
#        else:
#            print(a, file=f, end="")
#            i += 1